In [7]:
import os
import time
import json
import glob

In [ ]:
for i in range(0,20):
    num = i+1
    with open("config1.json", "r") as f:
        config = json.load(f)
    config["target"]=f'batch_{num}'
    liglist = glob.glob("./indata/pdbqt/{}/*.pdbqt".format(config["target"]))
    with open("{}_ligands.txt".format(config["target"]), "w") as f:
        f.write(" ".join(liglist))
    cmd = "unidock --receptor {} ".format("./indata/{}.pdbqt".format("double"))
    cmd += "--ligand_index {} ".format("{}_ligands.txt".format(config["target"]))
    cmd += "--center_x {:.2f} --center_y {:.2f} --center_z {:.2f} ".format(
        config["center_x"], config["center_y"], config["center_z"])
    cmd += "--size_x 30 --size_y 30 --size_z 30 "
    cmd += "--dir ./result/{} ".format(config["target"])
    cmd += "--exhaustiveness {} ".format(config["nt"])
    cmd += "--max_step {} ".format(config["ns"])
    cmd += "--num_modes 9  --scoring {} --refine_step {} ".format(config["sf"], config.get("rs", 5))
    cmd += "--seed {}".format(config.get("seed", 42))
    os.makedirs("result/{}".format(config["target"]), exist_ok=True)
    with open("rundock.sh", "w") as f:
        f.write(cmd)
    os.system("echo 'costtime'>> result/costtime.csv")
    st = time.time()
    os.system("bash rundock.sh")
    os.system("echo '{}'>> result/costtime.csv".format(time.time()-st))
    for _,_,fns in os.walk("./result/{}".format(config["target"])):
        break
    csv_name = "./result/{}_{}_nt{}_ns{}_seed{}_{}.csv".format(config["target"], config["sf"], config["nt"], config["ns"], config["seed"], config.get("gpu_type", 'c12_m92_1 * NVIDIA V100').split()[-1])
    
    for fn in fns:
        with open("result/{}/{}".format(config["target"], fn), "r") as f:
            f.readline()
            score = float(f.readline().split()[3])
            idx = int(fn.split("_")[0].replace("mol", ""))
            os.system("echo '{},{},{}'>> {}".format(idx, "mol", score, csv_name))
    